<center><img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300"></center><br/>

# Assignment: Notebook for Peer Assignment

Estimated time needed: 60 minutes


# Assignment Scenario

Congratulations! You have just been hired by a US Weather forecast firm as a data scientist.

The company is considering the weather condition to help predict the possibility of precipitations, which involves using various local climatological variables, including temperature, wind speed, humidity, dew point, and pressure. The data you will be handling was collected by a NOAA weather station located at the John F. Kennedy International Airport in Queens, New York.

Your task is to provide a high level analysis of weather data in JFK Airport. Your stakeholders want to understand the current and historical record of precipitations based on different variables. For now they are mainly interested in a macro-view of JFK Airport Weather, and how it relates to the possibility to rain because it will affect flight delays and etc.


# Introduction

This project relates to the NOAA Weather Dataset - JFK Airport (New York). The original dataset contains 114,546 hourly observations of 12 local climatological variables (such as temperature and wind speed) collected at JFK airport. This dataset can be obtained for free from the IBM Developer [Data Asset Exchange](https://developer.ibm.com/exchanges/data/all/jfk-weather-data/). 

For this project, you will be using a subset dataset, which contains 5727 rows (about 5% or original rows) and 9 columns. The end goal will be to predict the precipitation using some of the available features. In this project, you will practice reading data files, preprocessing data, creating models, improving models and evaluating them to ultimately choose the best model.




## Table of Contents:

Using this R notebook you will complete **10 tasks**:
* [0. Import Modules](#cell0)
* [1. Download and Unzip NOAA Weather Dataset](#cell1)
* [2. Read Dataset into Project](#cell2)
* [3. Select Subset of Columns](#cell3)
* [4. Clean Up Columns](#cell4)
* [5. Convert Columns to Numerical Types](#cell5)
* [6. Rename Columns](#cell6)
* [7. Exploratory Data Analysis](#cell7)
* [8. Linear Regression](#cell8)
* [9. Improve the Model](#cell9)
* [10. Find Best Model](#cell10)


<a id="cell0"></a>
## 0. Import required modules

Tidymodels is a collection of packages that use tidyverse principles to easily do the entire modeling process from preprocessing initial data, to creating a model, to tunning hyperparameters. The tidymodels packages can be used to produce high quality statistical and machine learning models. Our Jupyter notebook platforms have a built-in Tidyverse, Tidymodels and rlang packages so we do not need to install these packages prior to loading library. However, if you decide to run this lab on your RStudio Desktop locally on your machine, you can remove the commented lines of code to install these packages before loading.
 


In [1]:
# Install tidymodels if you haven't done so
install.packages("rlang")
install.packages("tidymodels")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Warning message:
“package ‘tidymodels’ is not available (for R version 3.5.1)”

**Note: After installing the packages, restart the kernel. Without installing the packages again, load them. Tidyverse and Tidymodels will be the two main packages you will use.**


In [2]:
# Library for modeling
library(tidymodels)

# Load tidyverse
library(tidyverse)

Warning message:
“replacing previous import ‘lifecycle::last_warnings’ by ‘rlang::last_warnings’ when loading ‘tibble’”Warning message:
“replacing previous import ‘ellipsis::check_dots_unnamed’ by ‘rlang::check_dots_unnamed’ when loading ‘tibble’”Warning message:
“replacing previous import ‘ellipsis::check_dots_used’ by ‘rlang::check_dots_used’ when loading ‘tibble’”Warning message:
“replacing previous import ‘ellipsis::check_dots_empty’ by ‘rlang::check_dots_empty’ when loading ‘tibble’”── Attaching packages ────────────────────────────────────── tidymodels 0.1.0 ──
✔ broom     0.5.6      ✔ recipes   0.1.12
✔ dials     0.0.6      ✔ rsample   0.0.5 
✔ dplyr     0.8.5      ✔ tibble    3.0.1 
✔ ggplot2   3.3.0      ✔ tune      0.1.0 
✔ infer     0.5.1      ✔ workflows 0.1.1 
✔ parsnip   0.1.0      ✔ yardstick 0.0.6 
✔ purrr     0.3.4      
── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard()  masks scales::discard()
✖ dplyr::filter()   masks


### Understand the Dataset

The original NOAA JFK dataset contains 114,546 hourly observations of various local climatological variables (including temperature, wind speed, humidity, dew point, and pressure). 

In this project you will use a sample dataset, which is around 293 KB. [Link to the sample dataset](https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/noaa-weather-sample-data.tar.gz).

The sample contains 5727 rows (about 5% or original rows) and 9 columns, which are:
- DATE
- HOURLYDewPointTempF
- HOURLYRelativeHumidity
- HOURLYDRYBULBTEMPF
- HOURLYWETBULBTEMPF
- HOURLYPrecip
- HOURLYWindSpeed
- HOURLYSeaLevelPressure
- HOURLYStationPressure

The original dataset is much bigger. Feel free to explore the original dataset. [Link to the original dataset.](https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/noaa_weather.html) 

For more information about the dataset, checkout the [preview](https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/data-preview/index.html?_ga=2.176781478.281508226.1616293518-1509963377.1616117067&cm_mc_uid=90945889198916153255549&cm_mc_sid_50200000=64650651616293516933) of NOAA Weather - JFK Airport.


<a id="cell1"></a>

## 1. Download NOAA Weather Dataset

Use the `download.file()` function to download the sample dataset from the URL below.

URL = 'https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/noaa-weather-sample-data.tar.gz'


In [9]:
url <- "https://dax-cdn.cdn.appdomain.cloud/dax-noaa-weather-data-jfk-airport/1.1.4/noaa-weather-sample-data.tar.gz"
download.file(url, destfile = "lax_to_jfk.tar.gz")

Untar the zipped file.


In [10]:
untar("lax_to_jfk.tar.gz", tar = "internal")

Warning message in untar2(tarfile, files, list, exdir, restore_times):
“using pax extended headers”

<a id="cell2"></a>
## 2. Extract and Read into Project
We start by reading in the raw dataset. You should specify the file name as "noaa-weather-sample-data/jfk_weather_sample.csv".


In [11]:
weather_data <- read_csv("noaa-weather-sample-data/jfk_weather_sample.csv",
                         col_types = cols('DATE' = col_number(),
                                          'HOURLYPrecip' = col_number()))

Warning message:
“393 parsing failures.
row          col expected actual                                              file
  5 HOURLYPrecip a number      T 'noaa-weather-sample-data/jfk_weather_sample.csv'
 29 HOURLYPrecip a number      T 'noaa-weather-sample-data/jfk_weather_sample.csv'
 41 HOURLYPrecip a number      T 'noaa-weather-sample-data/jfk_weather_sample.csv'
 49 HOURLYPrecip a number      T 'noaa-weather-sample-data/jfk_weather_sample.csv'
 64 HOURLYPrecip a number      T 'noaa-weather-sample-data/jfk_weather_sample.csv'
... ............ ........ ...... .................................................
See problems(...) for more details.
”

Next, display the first few rows of the dataframe.


In [12]:
head(weather_data)

DATE,HOURLYDewPointTempF,HOURLYRelativeHumidity,HOURLYDRYBULBTEMPF,HOURLYWETBULBTEMPF,HOURLYPrecip,HOURLYWindSpeed,HOURLYSeaLevelPressure,HOURLYStationPressure
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2015,60,46,83,68,0,13,30.01,29.99
2016,34,48,53,44,0,6,30.05,30.03
2013,33,89,36,35,0,13,30.14,30.12
2011,18,48,36,30,0,14,29.82,29.80
2015,27,61,39,34,NA,11,NA,30.50
2013,35,79,41,38,0,6,29.94,29.92


Also, take a `glimpse` of the dataset to see the different column data types and make sure it is the correct subset dataset with about 5700 rows and 9 columns.


In [13]:
glimpse(weather_data)

Rows: 5,727
Columns: 9
$ DATE                   <dbl> 2015, 2016, 2013, 2011, 2015, 2013, 2014, 2014…
$ HOURLYDewPointTempF    <dbl> 60, 34, 33, 18, 27, 35, 4, 14, 51, 71, 76, 19,…
$ HOURLYRelativeHumidity <dbl> 46, 48, 89, 48, 61, 79, 51, 65, 90, 94, 79, 37…
$ HOURLYDRYBULBTEMPF     <dbl> 83, 53, 36, 36, 39, 41, 19, 24, 54, 73, 83, 44…
$ HOURLYWETBULBTEMPF     <dbl> 68, 44, 35, 30, 34, 38, 15, 21, 52, 72, 78, 35…
$ HOURLYPrecip           <dbl> 0.00, 0.00, 0.00, 0.00, NA, 0.00, 0.00, 0.00, …
$ HOURLYWindSpeed        <dbl> 13, 6, 13, 14, 11, 6, 0, 11, 11, 5, 21, 7, 17,…
$ HOURLYSeaLevelPressure <dbl> 30.01, 30.05, 30.14, 29.82, NA, 29.94, 30.42, …
$ HOURLYStationPressure  <dbl> 29.99, 30.03, 30.12, 29.80, 30.50, 29.92, 30.4…


<a id="cell3"></a>
## 3. Select Subset of Columns

The end goal of this project will be to predict `HOURLYprecip` (precipitation) using a few other variables. Before you can do this, you first need to preprocess the dataset. Section 3 to section 6 focuses on preprocessing.

The first step in preprocessing is to select a subset of data columns and inspect the column types.

The key columns that we will explore in this project are:
- HOURLYRelativeHumidity
- HOURLYDRYBULBTEMPF
- HOURLYPrecip
- HOURLYWindSpeed
- HOURLYStationPressure

Data Glossary:
- 'HOURLYRelativeHumidity' is the relative humidity given to the nearest whole percentage.
- 'HOURLYDRYBULBTEMPF' is the dry-bulb temperature and is commonly used as the standard air temperature reported. It is given here in whole degrees Fahrenheit.
- 'HOURLYPrecip' is the amount of precipitation in inches to hundredths over the past hour. For certain automated stations, precipitation will be reported at sub-hourly intervals (e.g. every 15 or 20 minutes) as an accumulated amount of all precipitation within the preceding hour. A “T” indicates a trace amount of precipitation.
- 'HOURLYWindSpeed' is the speed of the wind at the time of observation given in miles per hour (mph).
- 'HOURLYStationPressure' is the atmospheric pressure observed at the station during the time of observation. Given in inches of Mercury (in Hg).

`Select` those five columns and store the modified dataframe as a new variable.


In [14]:
sub_weather_data <- weather_data %>%
    select(c(HOURLYRelativeHumidity,
             HOURLYDRYBULBTEMPF,
             HOURLYPrecip,
             HOURLYWindSpeed,
             HOURLYStationPressure))

Show the first 10 rows of this new dataframe.


In [15]:
head(sub_weather_data, 10)

HOURLYRelativeHumidity,HOURLYDRYBULBTEMPF,HOURLYPrecip,HOURLYWindSpeed,HOURLYStationPressure
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
46,83,0.00,13,29.99
48,53,0.00,6,30.03
89,36,0.00,13,30.12
48,36,0.00,14,29.80
61,39,NA,11,30.50
79,41,0.00,6,29.92
51,19,0.00,0,30.40
65,24,0.00,11,30.35
90,54,0.06,11,30.03


<a id="cell4"></a>
## 4. Clean Up Columns

From the dataframe preview above, we can see that the column `HOURLYPrecip` - which is the hourly measure of precipitation levels - contains both `NA` and `T` values. `T` specifies *trace amounts of precipitation* (meaning essentially no precipitation), while `NA` means *not available*, and is used to denote missing values. Additionally, some values also have "s" at the end of them, indicating that the precipitation was snow. 

Inspect the unique values present in the column `HOURLYPrecip` (with `unique(dataframe$column)`) to see these values.


In [16]:
unique(sub_weather_data$HOURLYPrecip)

[1] 0.00   NA 0.06 0.03 0.02 0.08 0.01 0.07 0.16 0.09 0.22 0.24 0.18 0.05 0.04
[16] 0.11 0.14 0.25 0.10 0.58 0.12 0.13 0.46 1.07 1.19 0.34 0.20 0.36 0.42 0.17
[31] 0.27 0.35 0.31 0.33 0.23 0.26 0.28 0.75 0.19 0.54 0.59 0.21

Having characters in values (like the "T" and "s" that you see in the unique values) will cause problems when you create a model because values for precipitation should be numerical. So you need to fix these values that have characters. 

Now, for the column `HOURLYPrecip`:
1. Replace all the `T` values with "0.0" and 
2. Remove "s" from values like "0.02s". In R, you can use the method `str_remove(column, pattern = "s$")` to remove the character "s" from the end of values. The "$" tells R to match to the end of values. The `pattern` is a regex pattern. Look at [here](https://www.rdocumentation.org/packages/stringi/versions/1.5.3/topics/about_search_regex) for more information about regex and matching to strings in R.

Remember that you can use `tidyverse`'s  `mutate()` to update columns.

You can check your work by checking if unique values of `HOURLYPrecip` still contain any `T` or `s`. Store the modified dataframe as a new variable.


In [17]:
unique(sub_weather_data$HOURLYPrecip)

[1] 0.00   NA 0.06 0.03 0.02 0.08 0.01 0.07 0.16 0.09 0.22 0.24 0.18 0.05 0.04
[16] 0.11 0.14 0.25 0.10 0.58 0.12 0.13 0.46 1.07 1.19 0.34 0.20 0.36 0.42 0.17
[31] 0.27 0.35 0.31 0.33 0.23 0.26 0.28 0.75 0.19 0.54 0.59 0.21

<a id="cell5"></a>
## 5. Convert Columns to Numerical Types
Now that you have removed the characters in the `HOURLYPrecip` column, you can safely covert the column to a numeric type.

First, check the types of the columns. You will notice that all are `dbl` (double or numeric) except for `HOURLYPrecip`, which is `chr` (character or string). Use the `glimpse` function from Tidyverse.


In [18]:
as.numeric(sub_weather_data$HOURLYPrecip)

[1] 0.00 0.00 0.00 0.00   NA 0.00 0.00 0.00 0.06   NA   NA 0.00 0.00 0.00
  [15] 0.00 0.00 0.00   NA 0.03   NA 0.03   NA 0.00 0.00 0.02   NA 0.00   NA
  [29]   NA 0.00 0.00   NA 0.00 0.00 0.00   NA 0.00 0.00 0.08   NA   NA   NA
  [43]   NA 0.00 0.00 0.00 0.01 0.00   NA   NA 0.00 0.00   NA 0.00 0.00   NA
  [57] 0.00 0.00 0.00 0.00 0.00   NA   NA   NA 0.00   NA 0.00 0.00   NA 0.00
  [71] 0.00 0.00 0.07   NA 0.00   NA 0.00 0.00   NA   NA 0.00   NA   NA 0.00
  [85] 0.00   NA 0.00   NA   NA 0.00   NA   NA 0.00 0.00   NA 0.00 0.16 0.00
  [99] 0.00 0.00 0.00 0.08 0.00 0.00 0.00   NA 0.00 0.00   NA   NA   NA   NA
 [113] 0.00   NA   NA 0.00 0.00 0.00 0.09 0.06   NA 0.00 0.00   NA   NA 0.00
 [127] 0.00   NA   NA 0.00 0.00 0.00 0.00 0.00   NA 0.00 0.00 0.00 0.00 0.00
 [141]   NA 0.00   NA 0.00   NA 0.22 0.00   NA 0.00   NA 0.02   NA   NA 0.00
 [155] 0.00 0.00   NA 0.00   NA   NA   NA   NA   NA   NA   NA 0.02 0.00 0.00
 [169] 0.00   NA   NA 0.00 0.00   NA 0.00 0.00 0.00   NA 0.01 0.00 0.00   NA
 [183]   NA 0.00   NA   NA 0.00   NA 0.00 0.00 0.00 0.00 0.24   NA 0.00 0.03
 [197]   NA 0.00   NA 0.00 0.00 0.00 0.00 0.00   NA 0.00   NA 0.00 0.00 0.00
 [211]   NA   NA 0.00   NA   NA 0.00 0.00 0.00   NA 0.00 0.00 0.00 0.00 0.00
 [225] 0.00   NA 0.00 0.00 0.00 0.03 0.00   NA   NA 0.00 0.00   NA 0.00   NA
 [239] 0.00   NA   NA   NA 0.00 0.00 0.00 0.00   NA 0.00 0.00   NA   NA   NA
 [253] 0.00 0.00 0.01   NA 0.00   NA 0.00 0.00 0.00   NA 0.00   NA   NA 0.00
 [267]   NA 0.00   NA 0.00 0.00 0.00   NA 0.00 0.00   NA   NA 0.00   NA   NA
 [281] 0.18 0.00 0.00 0.00   NA 0.00 0.00   NA 0.00 0.03 0.00   NA   NA   NA
 [295] 0.00 0.00   NA   NA 0.05 0.00 0.03 0.00 0.00 0.00 0.00 0.00 0.00   NA
 [309]   NA 0.00   NA 0.00   NA 0.00 0.00 0.00 0.00 0.00 0.00   NA   NA 0.00
 [323] 0.00   NA 0.00 0.00 0.00   NA 0.00   NA   NA   NA   NA 0.00 0.00 0.00
 [337] 0.06 0.00 0.00   NA   NA   NA 0.00 0.00 0.00 0.00   NA 0.00 0.00   NA
 [351]   NA   NA   NA 0.00   NA 0.00 0.00 0.01 0.00   NA 0.06 0.00   NA 0.00
 [365] 0.00 0.04 0.09 0.00 0.00 0.00 0.00 0.00 0.00   NA   NA   NA   NA 0.00
 [379]   NA 0.00 0.00   NA   NA 0.00   NA 0.00 0.00 0.00 0.00   NA   NA   NA
 [393] 0.00   NA 0.00 0.00 0.00 0.00 0.00 0.00   NA 0.00 0.00 0.07   NA   NA
 [407] 0.11 0.00 0.00   NA   NA 0.00 0.00 0.00 0.00 0.00   NA   NA 0.00   NA
 [421] 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00   NA 0.00 0.00   NA 0.00 0.00
 [435]   NA 0.00   NA 0.00 0.00 0.00 0.00   NA 0.00   NA   NA 0.00 0.00 0.00
 [449]   NA   NA   NA 0.00 0.00   NA   NA 0.00 0.00   NA 0.00 0.00   NA   NA
 [463] 0.00 0.09   NA 0.00 0.00   NA   NA   NA 0.06   NA 0.00   NA 0.00   NA
 [477] 0.00 0.00 0.00 0.01   NA 0.00 0.01 0.00 0.00 0.00 0.00 0.00 0.00 0.00
 [491] 0.00   NA 0.00 0.00   NA   NA 0.00 0.00 0.00 0.00   NA 0.00 0.00 0.00
 [505]   NA   NA   NA   NA 0.01 0.00 0.00 0.00   NA 0.00 0.00 0.00   NA   NA
 [519] 0.00 0.00 0.00 0.01 0.00   NA 0.00   NA 0.00   NA 0.00 0.00 0.00 0.00
 [533] 0.00   NA 0.00   NA 0.00 0.00   NA   NA   NA 0.00 0.00   NA 0.00 0.00
 [547]   NA 0.00 0.00   NA 0.00 0.05 0.00   NA   NA 0.00   NA   NA 0.00   NA
 [561] 0.00 0.00 0.00 0.00 0.00 0.00   NA 0.00 0.00   NA 0.00 0.00 0.05 0.00
 [575]   NA 0.00 0.00 0.00   NA 0.00 0.00 0.00 0.00   NA   NA   NA   NA   NA
 [589]   NA 0.00   NA   NA   NA 0.00 0.00   NA 0.00 0.00 0.00 0.00   NA 0.00
 [603] 0.00   NA 0.01 0.00   NA 0.00   NA 0.07   NA 0.00 0.00   NA 0.00   NA
 [617]   NA 0.00 0.00 0.00   NA 0.00 0.00 0.00 0.00   NA   NA 0.05   NA 0.00
 [631]   NA   NA   NA   NA   NA   NA 0.00 0.00 0.00 0.00 0.04 0.06 0.00 0.00
 [645] 0.00 0.00   NA 0.00 0.00 0.00 0.00   NA 0.00 0.00 0.00 0.00 0.00 0.00
 [659] 0.00 0.00   NA 0.00 0.00 0.00 0.00 0.00 0.00   NA 0.00 0.00 0.00 0.00
 [673] 0.00 0.00   NA 0.00 0.00   NA 0.00   NA 0.00 0.00 0.00 0.00 0.00   NA
 [687]   NA 0.00   NA 0.00 0.00 0.00 0.00 0.14 0.00   NA 0.00 0.00 0.00 0.00
 [701]   NA 0.00 0.00   NA 0.00 0.00 0.00 0.00 0.25   NA 0.06   NA 0.00 0.00
 [715] 0.00 0.00   NA 0.00   NA 0.00 0.06 0.00 0.00 0.08 0.00 0.00   NA   NA

Convert `HOURLYPrecip` to the `numeric` type and store the cleaned dataframe as a new variable.


In [20]:
new_w_data <- sub_weather_data %>%
    select(c(HOURLYRelativeHumidity,
             HOURLYDRYBULBTEMPF,
             as.numeric(sub_weather_data$HOURLYPrecip),
             HOURLYWindSpeed,
             HOURLYStationPressure))

ERROR: Error:
! Must subset columns with a valid subscript vector.
x Lossy cast from <double> to <integer>.
Caused by error in `stop_vctrs()`:
! Lossy cast from <double> to <integer>.
* Locations: 9, 19, 21, 25, 39, 47, 73, 97, 102, 119, 120, 146, 151, 166, 179, ...


<a id="cell6"></a>
## 6. Rename Columns
Let's rename the following columns as:
- 'HOURLYRelativeHumidity' to 'relative_humidity'
- 'HOURLYDRYBULBTEMPF' to 'dry_bulb_temp_f'
- 'HOURLYPrecip' to 'precip'
- 'HOURLYWindSpeed' to 'wind_speed'
- 'HOURLYStationPressure' to 'station_pressure'

You can use `dplyr::rename()`. Then, store the final dataframe as a new variable.


In [21]:
wea_data3 <- weather_data %>%
                replace_na(list(HOURLYPrecip = 0)) %>%
                rename("relative_humidity" = "HOURLYRelativeHumidity",
                       "dry_bulb_temp_f" = "HOURLYDRYBULBTEMPF",
                       "precip" = "HOURLYPrecip",
                       "wind_speed" = "HOURLYWindSpeed",
                       "station_pressure" = "HOURLYStationPressure")

In [22]:
head(wea_data3)

DATE,HOURLYDewPointTempF,relative_humidity,dry_bulb_temp_f,HOURLYWETBULBTEMPF,precip,wind_speed,HOURLYSeaLevelPressure,station_pressure
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2015,60,46,83,68,0,13,30.01,29.99
2016,34,48,53,44,0,6,30.05,30.03
2013,33,89,36,35,0,13,30.14,30.12
2011,18,48,36,30,0,14,29.82,29.80
2015,27,61,39,34,0,11,NA,30.50
2013,35,79,41,38,0,6,29.94,29.92


In [23]:
wea_data3 <- wea_data3 %>%
                replace_na(list(relative_humidity = 0,
                                dry_bulb_remp_f = 0,
                                precip = 0,
                                wind_speed = 0,
                                station_pressure = 0))

In [24]:
glimpse(wea_data3)

Rows: 5,727
Columns: 9
$ DATE                   <dbl> 2015, 2016, 2013, 2011, 2015, 2013, 2014, 2014…
$ HOURLYDewPointTempF    <dbl> 60, 34, 33, 18, 27, 35, 4, 14, 51, 71, 76, 19,…
$ relative_humidity      <dbl> 46, 48, 89, 48, 61, 79, 51, 65, 90, 94, 79, 37…
$ dry_bulb_temp_f        <dbl> 83, 53, 36, 36, 39, 41, 19, 24, 54, 73, 83, 44…
$ HOURLYWETBULBTEMPF     <dbl> 68, 44, 35, 30, 34, 38, 15, 21, 52, 72, 78, 35…
$ precip                 <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00…
$ wind_speed             <dbl> 13, 6, 13, 14, 11, 6, 0, 11, 11, 5, 21, 7, 17,…
$ HOURLYSeaLevelPressure <dbl> 30.01, 30.05, 30.14, 29.82, NA, 29.94, 30.42, …
$ station_pressure       <dbl> 29.99, 30.03, 30.12, 29.80, 30.50, 29.92, 30.4…


<a id="cell7"></a>
## 7. Exploratory Data Analysis
Now that you have finished preprocessing the dataset, you can can start exploring the columns more.

First, split the data into a training and testing set. Splitting a dataset is done randomly, so to have reproducible results set the seed = 1234. Also, use 80% of the data for training.


Next, looking at just the **training set**, plot histograms or box plots of the variables (`relative_humidity`, `dry_bulb_temp_f`, `precip`, `wind_speed`,  `station_pressure`) for an intial look of their distributions using `tidyverse`'s `ggplot`. Leave the testing set as is because it is good practice to not see the testing set until evaluating the final model.


<a id="cell8"></a>
## 8. Linear Regression 
After exploring the dataset more, you are now ready to start creating models to predict the precipitation (`precip`).

Create simple linear regression models where `precip` is the response variable and each of `relative_humidity`, `dry_bulb_temp_f`,`wind_speed` or `station_pressure` will be a predictor variable, e.g. `precip ~ relative_humidity`, `precip ~ dry_bulb_temp_f`, etc. for a total of four simple models. 
Additionally, visualize each simple model with a scatter plot.


<a id="cell9"></a>
## 9. Improve the Model
Now, try improving the simple models you created in the previous section. 

Create at least two more models, each model should use at least one of the different techniques:
1. Add more features/predictors
2. Add regularization (L1, L2 or a mix)
3. Add a polynomial component

Also, for each of the models you create, check the model performance using the **training set** and a metric like MSE, RMSE, or R-squared.

Consider using `tidymodels` if you choose to add regularization and tune lambda.


<a id="cell10"></a>
## 10. Find Best Model
Compare the regression metrics of each model from section 9 to find the best model overall. To do this, 

1. Evaluate the models on the **testing set** using at least one metric (like MSE, RMSE or R-squared).
2. After calculating the metrics on the testing set for each model, print them out in as a table to easily compare. You can use something like:
```
model_names <- c("model_1", "model_2", "model_3")
train_error <- c("model_1_value", "model_2_value", "model_3_value")
test_error <- c("model_1_value", "model_2_value", "model_3_value")
comparison_df <- data.frame(model_names, train_error, test_error)
```
3. Finally, from the comparison table you create, conclude which model performed the best.


In [25]:
model_names <- c("precip ~ relative_humidity", "precip ~ dry_bulb_temp_f", "precip ~ wind_speed", "precip ~ station_pressure", "Multiple Linear Regression", "Ridge (L2) regularization", "Elastic Net Regularization")
sqr <- c("0.02471808", "0.0002962591", "0.003871889", "0.0002849881", "0.03841775", "0.03022052", "0.03805498")
comparision_df <- data.frame(model_names, sqr)
comparision_df

model_names,sqr
<fct>,<fct>
precip ~ relative_humidity,0.02471808
precip ~ dry_bulb_temp_f,0.0002962591
precip ~ wind_speed,0.003871889
precip ~ station_pressure,0.0002849881
Multiple Linear Regression,0.03841775
Ridge (L2) regularization,0.03022052
Elastic Net Regularization,0.03805498


## Author(s)

<h4> Yiwen Li </h4>

## Contributions

<h4> Tiffany Zhu </h4>

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
